In [1]:
# Part a

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_2000 = pd.read_csv("hw4_nasdaq00.txt", names=["price"])
df_2000 = df_2000.price/ df_2000.price.max() # Normalize on max to keep everything in positive quadrant

In [4]:
def log_prob(x, a):
    """
    :param x double[4]
    :param a double[3]
    """
    prob = 1 / np.sqrt(2 * np.pi) * np.exp(-.5 * (x[3]- a[0] * x[2] -a[1] *x[1] - a[2] * x[0]) ** 2)
    return np.log(prob)

def func(a):
    buff = df_2000.rolling(window=4).apply(lambda x: log_prob(x, a), raw=True).dropna()
    return sum(buff)

def minimize_func(a):
    return - func(a)

In [5]:
# Create data strides
def stride_data(data):
    a_strides = np.lib.stride_tricks.as_strided(data, 
                                    shape=(len(data), 3),
                                    strides=(8,8)
                                   )[:-3]

    b_strides = data[3:].to_numpy().reshape(-1,1)
    return a_strides, b_strides
a_strides, b_strides = stride_data(df_2000)

# Compute A matrix and b vector
A = np.zeros((3,3))
b = np.zeros((3,1))
for i in range(3, len(a_strides)):
    A += a_strides[i-3:i]
    b += b_strides[i-3:i]
    
A /= (len(a_strides) - 3)
b /= (len(a_strides) - 3)

# Compute weights Aw = b --> inv(A)b
weights = np.matmul(
    np.linalg.inv(A), b
).reshape(1,-1)[0]

log_likelihood = func(weights)
print(f"Weights: {weights}")
print(f"Log Likelihood: {log_likelihood}")

Weights: [ 0.59827443 -0.33076581  0.72952856]
Log Likelihood: -226.16589322164688


In [6]:
# Checking 
from scipy.optimize import least_squares
initial = np.array([1,1,1])
final = least_squares(minimize_func, initial)
print(f"Weights: {final.x}")
print(f"Log Likelihood: {-final.fun[0]}")
print("Pretty close log likelihood")

Weights: [0.41870383 0.31205593 0.26676497]
Log Likelihood: -226.14794179168985
Pretty close log likelihood


# Part b

In [7]:
a = np.matrix(a_strides)
w = np.matrix(weights)
print("MSE for 2000")
np.mean(
    np.array(
        np.matmul(a, np.transpose(w))  - b_strides
    ) ** 2
)

MSE for 2000


0.0007425384495222976

In [8]:
df_2001 = pd.read_csv("hw4_nasdaq01.txt", names=["price"])
df_2001 = df_2001.price/ df_2001.price.max()
a_strides, b_strides = stride_data(df_2001)
a = np.matrix(a_strides)
w = np.matrix(weights)
print("MSE for 2001")
np.mean(
    np.array(
        np.matmul(a, np.transpose(w))  - b_strides
    ) ** 2
)

MSE for 2001


0.0004951445800134974